## **Model Inference**
Pada tahap ini, saya akan menggunakan model dan komponen preprocessing yang sudah disimpan pada tahap sebelumnya untuk membuat prediksi nasabah baru ini masuk ke dalam segmentasi/cluster mana. 

In [79]:
# Import Libraries
import pickle
import pandas as pd
import numpy as np

# Mengabaikan warning
import warnings
warnings.filterwarnings('ignore')

**1. Load model**


In [80]:
with open('kproto_bundle.pkl', 'rb') as f:
    model_bundle = pickle.load(f)

kproto = model_bundle['kproto_model']
pca = model_bundle['pca_model']
scaler = model_bundle['scaler']
numerical_cols = model_bundle['numerical_cols']
categorical_cols = model_bundle['categorical_cols']
final_columns_order = model_bundle['final_columns_order']
categorical_indices = model_bundle['categorical_indices']  

**2. Mengisi data user baru**

In [81]:
# Pastikan semua kolom yang ada di komponen preprocessing ada di sini dengan nama yang sama persis seperti saat training.
# Data user baru
new_user_raw = pd.DataFrame([{
        'call_failure': 0, # Pastikan nama kolom ini sama persis dengan saat training
        'subscription_length': 37,
        'seconds_of_use': 875,
        'frequency_of_use': 14,
        'frequency_of_sms': 0,
        'distinct_called_numbers': 11,
        'customer_value': 40.005,
        'charge_amount': 0,
        'age_group': 2,
        'complains': 0,
        'tariff_plan': 1,
        'status': 2,
        'age':25,
}])
new_user = new_user_raw[final_columns_order]
new_user

,call_failure,complains,subscription_length,charge_amount,age_group,tariff_plan,status,customer_value
0,0,0,37,0,2,1,2,40.005


**3. Preprocessing & Predict untuk Inference**

In [82]:
# Preprocessing
numerical_data = new_user[numerical_cols]
categorical_data = new_user[categorical_cols].astype(str)

# Transform
num_scaled = scaler.transform(numerical_data)
num_pca = pca.transform(num_scaled)

# Gabungkan
final_input = np.hstack([num_pca, categorical_data.to_numpy()])

# Debug shape dan tipe (sementara aja)
print("Final input shape:", final_input.shape)
print("Categorical indices:", categorical_indices)

# Predict
predicted_cluster = kproto.predict(final_input, categorical=categorical_indices)
print(f"Data pelanggan diprediksi masuk ke dalam Klaster: {predicted_cluster[0]}")

# Tambahkan ke dataframe
new_user['cluster'] = predicted_cluster
new_user.T

Final input shape: (1, 8)
Categorical indices: [5, 6, 7]
Data pelanggan diprediksi masuk ke dalam Klaster: 1


,0
call_failure,0.000
complains,0.000
subscription_length,37.000
charge_amount,0.000
age_group,2.000
tariff_plan,1.000
status,2.000
customer_value,40.005
cluster,1.000
